# Content-based Filtering

Aproach:
- Table mit Features und Ratings erstellen, ohne User - auf Basis des OMDB Datasets

- sklearn-kNN mit cosine-similarity darauf anwenden

- Funktion schreiben, die auf basis des ratings der neighbours das rating eines Filmes vorhersagt.

## This is a WIP: Decided to do omdb data cleaning first - bevore a useful model can be applied

Sources:

#### https://heartbeat.fritz.ai/recommender-systems-with-python-part-i-content-based-filtering-5df4940bd831

#### https://www.kaggle.com/johnwill225/movie-recommendations

#### https://towardsdatascience.com/how-we-built-a-content-based-filtering-recommender-system-for-music-with-python-c6c3b1020332


In [1]:
import pandas as pd
import numpy as np
import string
import math
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA
kwargs = dict(random_state=42)

In [2]:
movies = pd.read_csv('../../data/preprocessed/movies_id_updated.csv')
average_ratings = pd.read_csv('../../data/preprocessed/average_ratings.csv')
omdb = pd.read_csv('../../data/preprocessed/omdb_cleaned.csv')
ratings = pd.read_csv('../../data/preprocessed/ratings_clean_std_0.csv')

In [3]:
movies.head()

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...


In [4]:
ratings.head()

,Unnamed: 0,user_id,imdbID,rating
0,0,1264,tt0047034,3.5
1,1,213,tt0304141,2.5
2,2,593,tt0369436,3.0
3,3,609,tt1077258,4.0
4,4,1590,tt0052182,4.0


In [5]:
omdb.head()

,Title,Year,Rated,Runtime,Writer,Plot,Language,imdbRating,imdbVotes,imdbID,...,Series,Released_season,Released_month,Released_day,PG_Rating,Available_languages,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,Toy Story,1995.0,G,81.0,"John Lasseter (original story by), Pete Docter...",A cowboy doll is profoundly threatened and jea...,['English'],8.3,852896.0,tt0114709,...,0,4.0,11.0,22.0,0.0,1,0.0,3.0,0.0,0.0
1,Jumanji,1995.0,PG,104.0,"Jonathan Hensleigh (screenplay by), Greg Taylo...",When two kids find and play a magical board ga...,"['English', ' French']",7.0,294340.0,tt0113497,...,0,1.0,12.0,15.0,1.0,2,0.0,0.0,0.0,0.0
2,Grumpy Old Men,1993.0,PG-13,103.0,Mark Steven Johnson,A lifelong feud between two neighbors since ch...,['English'],7.0,41401.0,tt0107050,...,0,1.0,12.0,25.0,2.0,1,0.0,0.0,0.0,0.0
3,Waiting to Exhale,1995.0,R,124.0,"Terry McMillan (novel), Terry McMillan (screen...","Based on Terry McMillan's novel, this film fol...",['English'],5.9,9222.0,tt0114885,...,0,1.0,12.0,22.0,3.0,1,0.0,0.0,0.0,0.0
4,Father of the Bride Part II,1995.0,PG,106.0,"Albert Hackett (screenplay ""Father's Little Di...",George Banks must deal not only with the pregn...,['English'],6.0,33005.0,tt0113041,...,0,1.0,12.0,8.0,1.0,1,0.0,0.0,0.0,1.0


In [6]:
omdb.columns

Index(['Title', 'Year', 'Rated', 'Runtime', 'Writer', 'Plot', 'Language',
       'imdbRating', 'imdbVotes', 'imdbID', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'Released_season', 'Released_month', 'Released_day',
       'PG_Rating', 'Available_languages', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated'],
      dtype='object')

In [7]:
# Select relevant columns
omdb = omdb[['imdbID', 'Year', 'Runtime', 'Language', 'imdbRating', 'imdbVotes', 'Rotten Tomatoes', 'Metacritic',
       'Series', 'PG_Rating', 'Oscars_won', 'Oscars_nominated',
       'Golden_globe_won', 'Golden_globe_nominated']]

In [8]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      9
Language                     0
imdbRating                   4
imdbVotes                    5
Rotten Tomatoes           1237
Metacritic                4019
Series                       0
PG_Rating                 2314
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [9]:
# Add movies which could not be retrieved by omdb
imdb_ids = pd.DataFrame(movies['imdbID'].unique()).rename(columns={0:'imdbID'})
omdb = imdb_ids.merge(omdb, how='left', on='imdbID')

In [10]:
# missing entries in omdb dataset
omdb[omdb['Year'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [11]:
omdb[omdb['Language'].isna()]['imdbID']

707     tt1233247
2111    tt0800175
2241    tt0134948
3399    tt0257001
3810    tt0266860
4822    tt0720339
5118    tt0313487
7581    tt0439663
8140    tt0383974
9411    tt0182989
Name: imdbID, dtype: object

In [12]:
indices = omdb[omdb['imdbRating'].isna()]['imdbID'].index

In [13]:
indices

Int64Index([707, 2111, 2241, 3399, 3810, 4822, 5118, 7581, 8140, 9411], dtype='int64')

In [14]:
# Dropping the 10 almost empty movies
for i in indices:
    omdb = omdb.drop([i],axis=0,)

In [15]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      5
Language                     0
imdbRating                   0
imdbVotes                    1
Rotten Tomatoes           1236
Metacritic                4015
Series                       0
PG_Rating                 2310
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [16]:
# Replace for Series, PG and awards NaN with 0 and handle accordingly 
for i in range (9,14):
    omdb.iloc[:,i] = omdb.iloc[:,i].fillna(0)

In [17]:
omdb.isna().sum()

imdbID                       0
Year                         0
Runtime                      5
Language                     0
imdbRating                   0
imdbVotes                    1
Rotten Tomatoes           1236
Metacritic                4015
Series                       0
PG_Rating                    0
Oscars_won                   0
Oscars_nominated             0
Golden_globe_won             0
Golden_globe_nominated       0
dtype: int64

In [18]:
# Comment Max: Vorschlag - Stattdessen Median/Modalwert benutzen benutzen
# Fill NaN Series
omdb.loc[:,'Series'] = omdb.loc[:,'Series'].fillna(0)
# Fill NaN Runtime
omdb.loc[:,'Runtime'] = omdb.loc[:,'Runtime'].fillna(omdb.loc[:,'Runtime'].median())
# Fill NaN for imdbVotes
omdb.loc[:,'imdbVotes'] = omdb.loc[:,'imdbVotes'].fillna(omdb.loc[:,'imdbVotes'].median())
#for i in range (4,16):
    #omdb.iloc[:,i] = omdb.iloc[:,i].fillna(omdb.iloc[:,i].median())

In [19]:
omdb['Rotten Tomatoes'].describe()

count    8177.000000
mean        6.223138
std         2.812948
min         0.000000
25%         4.000000
50%         6.800000
75%         8.600000
max        10.000000
Name: Rotten Tomatoes, dtype: float64

In [20]:
omdb['Metacritic'].describe()

count    5398.000000
mean        5.779918
std         1.842826
min         0.100000
25%         4.500000
50%         5.900000
75%         7.100000
max        10.000000
Name: Metacritic, dtype: float64

In [21]:
omdb['Rotten Tomatoes'].isna()

0       False
1       False
2       False
3       False
4       False
        ...  
9418    False
9419    False
9420    False
9421    False
9422     True
Name: Rotten Tomatoes, Length: 9413, dtype: bool

In [22]:
omdb = omdb.rename(columns={"Rotten Tomatoes": "RottenTomatoes"})

In [23]:
omdb.loc[9422, 'RottenTomatoes']

nan

In [24]:
omdb.loc[9421, 'RottenTomatoes']

5.5

In [25]:
omdb['RottenTomatoes'].describe()

count    8177.000000
mean        6.223138
std         2.812948
min         0.000000
25%         4.000000
50%         6.800000
75%         8.600000
max        10.000000
Name: RottenTomatoes, dtype: float64

In [26]:
omdb.loc[0, 'RottenTomatoes']

10.0

In [27]:
# Replace RT Score with Metacritic if NaN
# where Replace values where the condition is False.
omdb['RottenTomatoes'] = omdb['RottenTomatoes'].where(~omdb['RottenTomatoes'].isna(),omdb['Metacritic'])

In [28]:
omdb.loc[0, 'RottenTomatoes']

10.0

In [29]:
omdb.loc[9422, 'RottenTomatoes']

nan

In [30]:
omdb.loc[9422, 'Metacritic']

nan

In [31]:
omdb.loc[9421, 'RottenTomatoes']

5.5

In [32]:
# Replace Metacritic with RT Scroe if NaN
omdb['Metacritic'].where(~omdb['Metacritic'].isna(),omdb['RottenTomatoes'])

0       9.5
1       3.9
2       5.3
3       5.6
4       4.9
       ... 
9418    6.8
9419    3.3
9420    8.0
9421    4.7
9422    NaN
Name: Metacritic, Length: 9413, dtype: float64

In [33]:
omdb['RottenTomatoes'].isna().sum()

1048

In [34]:
omdb['Metacritic'].isna().sum()

4015

In [35]:
# Fill remaining with mean()
omdb['RottenTomatoes']=omdb['RottenTomatoes'].where(~omdb['RottenTomatoes'].isna(),omdb['RottenTomatoes'].mean())
omdb['Metacritic']=omdb['Metacritic'].where(~omdb['Metacritic'].isna(),omdb['Metacritic'].mean())

In [36]:
omdb['Metacritic'].isna().sum()

0

In [37]:
omdb['RottenTomatoes'].isna().sum()

0

In [38]:
merged_data = ratings.merge(omdb, how='left', on='imdbID')

In [39]:
merged_data.head()

,Unnamed: 0,user_id,imdbID,rating,Year,Runtime,Language,imdbRating,imdbVotes,RottenTomatoes,Metacritic,Series,PG_Rating,Oscars_won,Oscars_nominated,Golden_globe_won,Golden_globe_nominated
0,0,1264,tt0047034,3.5,1954.0,96.0,['Japanese'],7.6,27485.0,9.3,7.800000,0.0,0.0,0.0,0.0,0.0,0.0
1,1,213,tt0304141,2.5,2004.0,142.0,['English'],7.9,524511.0,9.0,8.200000,0.0,1.0,0.0,2.0,0.0,0.0
2,2,593,tt0369436,3.0,2008.0,88.0,['English'],5.7,61335.0,2.4,4.100000,0.0,2.0,0.0,0.0,0.0,0.0
3,3,609,tt1077258,4.0,2007.0,105.0,['English'],7.1,196149.0,7.4,5.779918,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1590,tt0052182,4.0,1958.0,100.0,['English'],7.4,6337.0,6.9,5.779918,0.0,0.0,2.0,0.0,0.0,0.0


In [40]:
merged_data = merged_data.drop(columns={'Unnamed: 0','Language'})

In [41]:
# Comment Max: No NaN rows anymore - except for language for the missing movies
merged_data.isna().sum()

user_id                     0
imdbID                      0
rating                      0
Year                      269
Runtime                   269
imdbRating                269
imdbVotes                 269
RottenTomatoes            269
Metacritic                269
Series                    269
PG_Rating                 269
Oscars_won                269
Oscars_nominated          269
Golden_globe_won          269
Golden_globe_nominated    269
dtype: int64

In [42]:
# convert imdbID from string to float
merged_data['imdbID'] = merged_data['imdbID'].str.replace(r'tt','')
merged_data['imdbID'] = merged_data['imdbID'].astype(float)
merged_data['imdbID'].unique()

array([ 47034., 304141., 369436., ...,  81433., 295480., 298072.])

In [46]:
#Statt    
    #Drop NaN rows - check later if it makes large difference!
#merged_data = merged_data.dropna()

# Jetzt einfach ma5 mean() eingefüllt
for i in range (3,17):
    merged_data.iloc[:,i] = merged_data.iloc[:,i].fillna(merged_data.iloc[:,i].median())

In [107]:
merged_data.isna().sum()

index                     0
user_id                   0
imdbID                    0
rating                    0
Year                      0
Runtime                   0
imdbRating                0
imdbVotes                 0
RottenTomatoes            0
Metacritic                0
Series                    0
PG_Rating                 0
Oscars_won                0
Oscars_nominated          0
Golden_globe_won          0
Golden_globe_nominated    0
dtype: int64

In [ ]:
# possibility to encode language as one-hot encoding
#mlb = MultiLabelBinarizer()
#mlb.fit_transform(merged_data['Language'])

In [ ]:
# TODO -> Restliches Script so umschreiben, dass imdbID aus der FeatureMatrix rausgeschmissen werden kann
# set index as imdbID in ascending order
#features = omdb.set_index('imdbID', drop = True)
#features = features.sort_index()

In [48]:
data_as_array = merged_data.to_numpy()

In [49]:
csr_data = csr_matrix(data_as_array.astype(np.float))
merged_data.reset_index(inplace=True)

In [50]:
k_neighbors = 10
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

In [51]:
print("Training")
knn.fit(csr_data)

Training


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [52]:
csr_data

<787541x15 sparse matrix of type '<class 'numpy.float64'>'
	with 8171057 stored elements in Compressed Sparse Row format>

In [53]:
# Do predictions
print("Do Prediction")
movie_idx = merged_data[merged_data['imdbID'] == 113497].index[0]
print(movie_idx)
distances, indices = knn.kneighbors(csr_data[movie_idx], n_neighbors=k_neighbors+1)
print(distances)
print(indices)

Do Prediction
3199
[[1.11022302e-16 6.28019858e-12 2.00961470e-11 2.00963690e-11
  4.64724925e-11 8.16405832e-11 9.16887677e-11 1.30625066e-10
  1.82121318e-10 3.26562333e-10 3.26562444e-10]]
[[  3199 619839 429437 237305 343809 332030 696703 397613  17982 621402
   84803]]


In [98]:
distances[0][0]

1.1102230246251565e-16

In [54]:
for i in range(0,indices.size):
    if i % 10 ==0:
        print(merged_data.iloc[[indices[0,i]]])

      index  user_id    imdbID  rating    Year  Runtime  imdbRating  \
3199   3199       42  113497.0     3.0  1995.0    104.0         7.0   

      imdbVotes  RottenTomatoes  Metacritic  Series  PG_Rating  Oscars_won  \
3199   294340.0             5.4         3.9     0.0        1.0         0.0   

      Oscars_nominated  Golden_globe_won  Golden_globe_nominated  
3199               0.0               0.0                     0.0  
       index  user_id    imdbID  rating    Year  Runtime  imdbRating  \
84803  84803       34  113497.0     4.0  1995.0    104.0         7.0   

       imdbVotes  RottenTomatoes  Metacritic  Series  PG_Rating  Oscars_won  \
84803   294340.0             5.4         3.9     0.0        1.0         0.0   

       Oscars_nominated  Golden_globe_won  Golden_globe_nominated  
84803               0.0               0.0                     0.0  


In [55]:
# zip bildet tuple der beiden Objekte)
ec_movie = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])[:0:-1]
  

In [56]:
for i in ec_movie:
    print(i)

(84803, 3.2656244375317556e-10)
(621402, 3.265623327308731e-10)
(17982, 1.8212131802641807e-10)
(397613, 1.306250663191122e-10)
(696703, 9.168876768939072e-11)
(332030, 8.164058318271827e-11)
(343809, 4.647249252087704e-11)
(237305, 2.0096369013344884e-11)
(429437, 2.009614696873996e-11)
(619839, 6.280198583397123e-12)


In [113]:
# function that computes a rating based on the neighbors
def compute_rating(neighbors, distances):
    # TODO: Function schreiben, die statt dem mean() der Nachbarn die Ratings auf Basis des Abstandes gewichtet
    # Weiter entfernte Filme werden geringer gewichtet nähere Filme werden stärker gewichtet
    
    for i in range(0,len(distances[0])):
        pred = (1-(distances[0][i]/distances.sum()))*neighbors['rating']
    
    #pred = np.mean(neighbors['rating'])
    
    return float(pred)

In [136]:
## function that predicts the rating of a movie from its imdbID and its 20 nearest neighbors

def predict_movie_rating(imdbID, userID, user_data):
    # get 20 (or x) nearest neighbors that were rated by userID
    # TODO: Use function from kNN.ipynb to find optimal k for algorithm 
    k_neighbors = 16    
    
    # Select all ratings given by User #userID
    ratings = user_data.loc[user_data['user_id'] == userID]
    
    # Get real rating -> remove this in the end -> currently done for validation
    real_ratings = ratings.loc[(ratings['imdbID'] == imdbID)]
    
    #remove real rating
    ratings = ratings[ratings['imdbID'] != imdbID] 
     
    
    if (ratings.to_numpy().size>0):   
        
        # Convert to Sparse Matrix Format
        #ratings_as_csr = csr_matrix(ratings)
        
        # Set algorithm and params 
        # TODO adapt neighbors based on number of given ratings by user_id ?
        knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=k_neighbors, n_jobs=-1)

        # Training
        #print('---- Training ConBF-kNN-Algorithm ----')
        #print('user_id: '+str(userID))
        #print('imdbID: '+str(imdbID))
        knn.fit(ratings)

        #Prediction -> get x nearest neighbors of imdbID
        distances , indices = knn.kneighbors(user_data.loc[user_data['imdbID'] == imdbID], n_neighbors=k_neighbors)
        
       
        # Collect neighbors by indices
        for i in range(0,indices.shape[1]):
            neighbors = ratings.iloc[[indices[0,i]]]
            
        # compute rating of movie(imbdID) based on the rating of the 20 nearest neighbors
        pred = compute_rating(neighbors, distances)
        
        #Generate Output for Understandability
        #print('Predicted Rating for '+str(imdbID)+': '+str(pred))
        #print('Real Rating of '+str(imdbID)+' was: '+ str(real_ratings['rating'].values[0]))
        
        # return rating prediction and real rating
        return pred , real_ratings['rating'].values[0]
        
    else:
         return "User has not rated other movies. Check input"
    

In [122]:
### Testing function ###
imdbID = 113627.0
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, merged_data)

Predicted Rating for 113627.0: 4.983103900759194
Real Rating of 113627.0 was: 5.0


In [123]:
### Testing function ###
imdbID = 145653
# Aufpassen userID und imdbID als float übergeben!!
userID = 123
pred , real = predict_movie_rating(imdbID, userID, merged_data)

Predicted Rating for 145653: 3.4481439294563994
Real Rating of 145653 was: 4.5


In [63]:
## DUPLICATED RATING!!! -> durch das umschreiben mit .values[0] wird das Doppelte Rating nicht mehr angezeigt -> tdm mal checken!

In [111]:
# Larger Test:
# TODO Refactoring needed #UglyPythonCode

def test_predict_mr(no_test_samples):
    # Sampling #no_test_samples of random samples from dataset for testing
    test_set = merged_data.sample(n=no_test_samples)
    
    predictions = pd.DataFrame(columns=['Prediction'])
    reals = pd.DataFrame(columns=['Real_Rating'])
    
    # Iterate over test-set and generate predicitons for it
    # TODO get rid of ugly for-loop
    for row in test_set.itertuples():
        imdbID = row.imdbID
        userID = row.user_id
        pred , real = predict_movie_rating(imdbID, userID, merged_data)
        predictions.loc[row.index] = pred
        reals.loc[row.index] = real
    
    rmse = mean_squared_error(reals['Real_Rating'], predictions['Prediction'], squared=False)
    print('RMSE: '+str(rmse))
    return float(rmse)

In [139]:
n = 50
test_predict_mr(n)

RMSE: 0.8982202650280295


0.8982202650280295

In [140]:
upper = 0
lower = 10

for i in range(0,10):
    n = 100
    rmse = test_predict_mr(n)
    
    if (float(rmse) < float(lower)):
        lower = rmse
    
    if (float(rmse) > float(upper)):
        upper = rmse
        
print('RMSE upper Bound: '+str(upper))
print('RMSE lower Bound: '+str(lower))

RMSE: 1.2051124004709146
RMSE: 1.3530600957644505
RMSE: 1.3287519619612989
RMSE: 1.2780180751575112
RMSE: 1.2437837398829847
RMSE: 1.1412267946764523
RMSE: 1.3845269303683074
RMSE: 1.1319490041349227
RMSE: 1.3490357591740214
RMSE: 1.171923103627717
RMSE upper Bound: 1.3845269303683074
RMSE lower Bound: 1.1319490041349227


In [141]:
n = 1000
test_predict_mr(n)

RMSE: 1.2247560110330107


1.2247560110330107